In [12]:
from torchsummary import summary
from python_lib.modules import *
import torch
import numpy as np
import os
from python_lib.saveasfile import *

In [13]:
input_feats = torch.rand([1,2,64]).to("cuda")
input_feats_np = input_feats.cpu().detach().numpy()
input_feats_np = np.reshape(input_feats_np, (2,64))
print(input_feats_np)
dim = input_feats_np.shape

flatten_inputs = input_feats_np.flatten()
with open(os.path.join("ECAPAweights", f"ecapainput.bin"), "wb") as f:
            # Write the dimensions down
            f.write(np.array(dim, dtype=np.int32).tobytes())
            # Write the flatten bias down
            f.write(flatten_inputs.tobytes())

[[0.75369895 0.47365534 0.04631549 0.57971156 0.49222088 0.6482848
  0.6341973  0.40538418 0.06989694 0.5507403  0.60797036 0.5018034
  0.78907204 0.1264118  0.03590226 0.41581845 0.31956482 0.31640375
  0.07271683 0.8352609  0.5753878  0.31308526 0.48919457 0.22090179
  0.04806745 0.25747478 0.6491883  0.6253344  0.72392887 0.6715279
  0.4152034  0.8586629  0.41310942 0.18586099 0.05589753 0.40654135
  0.7927932  0.14876276 0.6241265  0.20615232 0.60735464 0.7453405
  0.49164325 0.44260287 0.09521008 0.8434055  0.06873977 0.41850448
  0.11352092 0.18141454 0.16695625 0.8584323  0.7359674  0.3960212
  0.06126249 0.85519814 0.5904064  0.52389294 0.4231736  0.60239846
  0.51771396 0.6465695  0.5198399  0.37611926]
 [0.04829127 0.9093083  0.7491111  0.00501996 0.78101456 0.22385031
  0.37785482 0.01673681 0.7843546  0.41122788 0.7546224  0.8538427
  0.08612823 0.34328443 0.63188034 0.538451   0.3305779  0.88262796
  0.09293908 0.5321195  0.14994687 0.8919201  0.06027186 0.14615977
  0.985

In [14]:
import torch.nn.functional as F

F.softmax(input_feats, dim = 2)

tensor([[[0.0206, 0.0156, 0.0102, 0.0173, 0.0159, 0.0185, 0.0183, 0.0145,
          0.0104, 0.0168, 0.0178, 0.0160, 0.0213, 0.0110, 0.0100, 0.0147,
          0.0133, 0.0133, 0.0104, 0.0223, 0.0172, 0.0133, 0.0158, 0.0121,
          0.0102, 0.0125, 0.0186, 0.0181, 0.0200, 0.0190, 0.0147, 0.0229,
          0.0147, 0.0117, 0.0103, 0.0146, 0.0214, 0.0112, 0.0181, 0.0119,
          0.0178, 0.0204, 0.0158, 0.0151, 0.0107, 0.0225, 0.0104, 0.0147,
          0.0109, 0.0116, 0.0115, 0.0229, 0.0202, 0.0144, 0.0103, 0.0228,
          0.0175, 0.0164, 0.0148, 0.0177, 0.0163, 0.0185, 0.0163, 0.0141],
         [0.0097, 0.0230, 0.0196, 0.0093, 0.0203, 0.0116, 0.0135, 0.0094,
          0.0203, 0.0140, 0.0197, 0.0218, 0.0101, 0.0131, 0.0174, 0.0159,
          0.0129, 0.0224, 0.0102, 0.0158, 0.0108, 0.0226, 0.0099, 0.0107,
          0.0248, 0.0116, 0.0111, 0.0159, 0.0152, 0.0115, 0.0223, 0.0098,
          0.0174, 0.0149, 0.0176, 0.0157, 0.0171, 0.0232, 0.0158, 0.0150,
          0.0155, 0.0205, 0.0118, 0.0

In [15]:
model = ECAPA_TDNN(input_size = 2, channels=[8,8,8,8,16], lin_neurons=6, device = "cuda").to("cuda")

model_ = BlockSave(model.return_layers(), "fullecapa", "ECAPAweights")
model_.save()

initialblock = BlockSave(model.blocks[0].return_layers(), "initialblock", "ECAPAweights")
initialblock.save()

seres2_1 = BlockSave(model.blocks[1].return_layers(), "seres2_1", "ECAPAweights")
seres2_1.save()
seres2_2 = BlockSave(model.blocks[2].return_layers(), "seres2_2", "ECAPAweights")
seres2_2.save()
seres2_3 = BlockSave(model.blocks[3].return_layers(), "seres2_3", "ECAPAweights")
seres2_3.save()

mfa = BlockSave(model.mfa.return_layers(), "mfa", "ECAPAweights")
mfa.save()

asp = BlockSave(model.asp.return_layers(), "asp", "ECAPAweights")
asp.save()

# asp_bn = BlockSave(model.asp_bn.return_layers(), "asp_bn", "ECAPAweights")
# asp_bn.save()

fc = BlockSave([model.final[1]], "fc", "ECAPAweights")
fc.save()

In [17]:
model(input_feats)

tensor([[[-1.4901e-08],
         [ 7.4506e-09],
         [ 0.0000e+00],
         [ 0.0000e+00],
         [-2.2352e-08],
         [-7.8231e-08],
         [-2.9802e-08],
         [ 3.7253e-09]]], device='cuda:0', grad_fn=<MeanBackward1>)
tensor([[[0.0000],
         [0.3120],
         [0.1469],
         [0.0000],
         [0.2502],
         [0.0677],
         [0.1094],
         [0.3239],
         [0.0000],
         [0.2753],
         [0.0000],
         [0.3054],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0483],
         [0.0000],
         [0.3145],
         [0.2334],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.1488],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.1784],
         [0.1203],
         [0.0000],
         [0.3197],
         [0.2059],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.2272],
         [0.0000],
    

tensor([[0.0559, 0.1844, 0.1900, 0.2514, 0.2227, 0.0956]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)

In [ ]:
input_feats.mean(dim=2, keepdim=True).shape

torch.Size([1, 16, 1])

In [ ]:
import math

def get_padding_elem(L_in: int, stride: int, kernel_size: int, dilation: int):
    """This function computes the number of elements to add for zero-padding.

    Arguments
    ---------
    L_in : int
    stride: int
    kernel_size : int
    dilation : int

    Returns
    -------
    padding : int
        The size of the padding to be added
    """
    if stride > 1:
        padding = [math.floor(kernel_size / 2), math.floor(kernel_size / 2)]

    else:
        L_out = (
            math.floor((L_in - dilation * (kernel_size - 1) - 1) / stride) + 1
        )
        padding = [
            math.floor((L_in - L_out) / 2),
            math.floor((L_in - L_out) / 2),
        ]
    return padding

get_padding_elem(64,1,5,1)

[2, 2]